## Gear Optimization Notebook for Epic Seven
#### E7_Py_Gear_Selector by ja-bru

##### Initialize Notebook Settings

In [ ]:
%run item_potential.py

In [ ]:
## Uncomment the line below & run this cell to update hero database when new characters are released
# %run api_get.py

In [ ]:
from datetime import datetime
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 40)
import numpy as np
import itertools
import pickle
import json
import yaml
from ipywidgets import widgets, VBox, HBox
from IPython.display import display

import setup as st
import fx_lib as fx
with open(r'../inp/character_inputs.yaml') as file:
    target_stats = yaml.load(file, Loader=yaml.FullLoader)
with open('../inp/master_data.json') as json_file:
    data = json.load(json_file)
import customer_ui_fx as ui

In [ ]:
df_items = pd.read_pickle('../outp/equip_potential.pkl')
df_items['reco'] = ''
df_items['start_loc'] = df_items['hero']

## Section 1:  Notebook set up
Description of Settings:

| Syntax | Description |
| :----------- | :----------- |
| <b>Manual_Selection:</b> |  Primarily used for run in terminal, if Selection = 1, the user is prompted with a refined gear list to choose from for each hero | 
| <b>Gear_Limit:</b> |  To improve run time, the top 'n' gears for EACH slot is used in optimization | 
| <b>Auto_Adj_Gear_Limit:</b> |  Adjusts the number of gear pieces selected based on combinations to optimize run time vs available selections |
| <b>Use_Broken_Sets:</b> |  Only complete sets are used in gear optimization {default: 0} |
| <b>No_Equipped_Gear:</b> |  Set to 1 {default}: does not use any equipped gear.  To include gear equipped on other heroes set to 0. |
| <b>Keep_Current_Gear:</b> |  Keeps any currently equipped gear on the hero |
| <b>Gear_Enhance:</b> |  Enhance gear to minimum level for stat selection/optimization |
| <b>Flat_Sub:</b> |  Weight flat stat values in substats {range 0.0-1.0, default:0.8} |
| <b>Flat_Main:</b> |  Weight value attributed to {Necklace, Ring, Boots} with flat main stats {range 0.0-1.0, default:0.5} |
| <b>Min_Hero_Lvl:</b> |  Default Hero level of either 50 or 60 for stat calculation when the hero is not included in input file |

##### You can edit these settings before you get started by uncommenting them and changing the value, then running the cell

In [ ]:
st.GEAR_LIMIT = 15
st.NO_EQUIPPED_GEAR = 1
st.IGNORE_FLAT_MAIN_STATS = 1
# st.MIN_LEVEL = 50
# st.GEAR_ENHANCE = 12
# st.FLAT_SUB = 0.8
# st.FLAT_MAIN = 0.5

In [ ]:
fx.verify_setup()

In [ ]:
## Here are the latest settings from character_inputs.yaml:
hero_order = fx.startup_msg1(target_stats)
j=0

If you want to adjust the heroes above, uncomment the below cell and manually update heroes for optimization and locked gear

In [ ]:
# hero_order = ['Faithless Lidica','Dizzy','Falconer Kluri','Cermia']
lock_gear = []

In [ ]:
df_hero, char_list = fx.hero_json_to_df(hero_order, data)
df_items = fx.startup_msg2(df_items, lock_gear)

## Section 2:  Hero specific settings
### SELECT HERO
The following section can be looped through for one hero at a time.  You can select the hero and customize some settings to deliver optimal gear selection.  Once recommended gear has been assigned to the hero, that gear will become locked and previously equipped gear will be released.  Data is saved at the end of each hero optimization and can be recovered in case of error when doing multiple heroes at once.

To recover data, uncomment and run the cell below:

In [ ]:
# df_items = pd.read_pickle('../outp/upd_items.pkl')

Specify the hero and review settings

In [ ]:
char, hero_target = fx.start_hero(hero_order[j], target_stats)
w0,s4,s2,w1,w2,w3,w4,ms0,r0 = ui.run_widgets(hero_target, char_list, [st.GEAR_ENHANCE,st.GEAR_LIMIT,st.KEEP_CURR_GEAR,st.IGNORE_FLAT_MAIN_STATS], char)

Optional settings to change:

In [ ]:
VBox([w1,w4,w2])

Save if settings were changed:

In [ ]:
char, hero_target = fx.start_hero(w1.value, target_stats)
st.GEAR_ENHANCE = w2.value
st.KEEP_CURR_GEAR = 1 if w4.value == True else 0

Select weighting of hero stats

In [ ]:
if w4.value: print("Hero current has gear equipped: ", df_items[df_items.hero == char].set.unique(), "and you've selected keep existing gear")
VBox([HBox([VBox([widgets.Label("Stat Weights"),w0]), VBox([widgets.Label("4 Piece Sets"),s4]), VBox([widgets.Label("2 Piece Sets"),s2])]),
    HBox([VBox([widgets.Label("Necklace Main Stat"),ms0[0]]), VBox([widgets.Label("Ring Main Stat"),ms0[1]]), VBox([widgets.Label("Boot Main Stat"),ms0[2]]), VBox([widgets.Label("Rank Order Gear by:"), r0])]) ])

In [ ]:
hero_target = ui.update_settings(hero_target, w0, s4, s2, ms0)

## Section 3:  Run Gear Combinations

In [ ]:
gear_comb_dict = fx.set_combo(fx.equip_optimizer_input(df_items, char, hero_target, hero_target['Main_Stats'], r0.value), fx.l4, fx.l2)  ## Output gear_comb_dict[ [set_nm] , [type] , [ID] ]
sc_output = fx.set_combination_iterate(gear_comb_dict, fx.set_4[fx.set_4.Set_Nm.isin(hero_target['include_sets'])].Set_Nm.values , fx.set_2[fx.set_2.Set_Nm.isin(hero_target['include_sets'])].Set_Nm.values, hero_target['Force_4Set'])

Calculates the stats for each combination of gear
The gear initially equipped on this hero is also pulled in for comparison.  Some of the hero's gear may have already been assigned to another hero during this optimization and cannot be selected for the desired output.

In [ ]:
sc_df, hero_with_gear = fx.final_gear_combos(sc_output, char)

In [ ]:
odf = fx.get_combo_stats(sc_df, df_hero, fx.mainst_sum(sc_df, df_items), fx.subst_sum(sc_df, df_items), \
                        fx.set_sum(sc_df), fx.bonus_eqp_sum(df_hero[df_hero.Name == char]), char, hero_target)
idx_reco, choice_df = fx.run_stat_reco(odf, hero_with_gear, hero_target)

In [ ]:
idx_text = 'Enter the index value for the gear combination with the hero stats you would like to apply:'
idx = widgets.BoundedIntText(value=idx_reco, min=0, max=len(odf)-2, description="Index:", disabled=False)
uniqueSets = (odf['Set_1'].dropna().append(odf['Set_2'].dropna()).append(odf['Set_3'].dropna())).unique()
s1,s2,w3 = ui.run_stat_selector(uniqueSets)
box1 = VBox(s1)
box2 = VBox(s2)
box3 = VBox([widgets.Label("Choose the stat/column to sort by:"), w3] )

## Section 4:  Choose Gear
### REVIEW GEAR OPTIONS

This table will display some of the top options based on the hero settings:

In [ ]:
choice_df[['Complete','Set_1','Set_2','Set_3','WW','ATK','HP','DEF','SPD','CRIT','CDMG','EFF','RES','Dmg_Rating','EHP']]

You can also interact with the stats to display the top options:

In [ ]:
HBox([box1,box2,box3])

In [ ]:
print_df = ui.print_gear_options(odf,s1,s2,w3)
print_df

#### SELECT FINAL GEAR SET

In [ ]:
print(idx_text)
print("Value 0 to",len(odf)-2)
display(idx)

In [ ]:
print("Confirming index number:",idx.value)

##### Save the results for this hero:

In [ ]:
df_items = ui.save_hero(df_items, odf.loc[idx.value], char)
j+=1
ui.save_final_data(df_items)
print("Step 4/4 Complete:  Gear chosen and saved")

#### You can now go back to Section 2 and start on the next hero, or finish here.  Data has been saved in both .csv and .json files.  You can also look at the following data table to see your changes

In [ ]:
delta= np.unique(df_items[~(df_items.hero == df_items.start_loc)].hero.values)
delta = delta[delta!='']
cols = ['hero','start_loc','reco','slot','set','level','rarity','enhance','mainStat','subStat1','subStat2','subStat3','subStat4','efficiency','max_eff','id','Type']
df_items[cols][df_items.hero.isin(delta)|df_items.start_loc.isin(delta)].sort_values(by=['hero','Type'], ascending=[False, True])

## End of optimization notebook